## clear topic 

In [1]:
import requests
import csv

def get_wikipedia_content(topic):
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "titles": topic,
        "prop": "extracts",
        "explaintext": True,
        "redirects": 1
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    pages = data['query']['pages']
    for page_id, page_data in pages.items():
        return page_data.get('extract')

def generate_topic_data_wikipedia(topics, n_per_topic=2000):
    all_data = []
    topic_data_count = {}
    for topic in topics:
        content = get_wikipedia_content(topic)
        if content:
            sentences = content.split(". ")
            samples = [sentence for sentence in sentences if len(sentence.split()) >= 10]
            samples = samples[:n_per_topic]
            topic_data_count[topic] = len(samples)
            all_data.extend((sentence, topic) for sentence in samples)
        else:
            print(f"Failed to retrieve content for {topic}")
            topic_data_count[topic] = 0
    return all_data, topic_data_count

topics = [
"quantum_mechanics", "organic_chemistry", "fluid_dynamics", "thermodynamics","differential_geometry",
"plate_tectonics", "cellular_respiration", "electromagnetic_theory", "nuclear_physics", "crystal_structure",
"evolution_theory", "chemical_bonding", "classical_mechanics", "molecular_biology", "statistical_mechanics",
"crystal_structure", "quantum_field_theory", "particle_physics", "astrophysics", "cosmology",   
"evolution_theory", "chemical_bonding", "classical_mechanics", "molecular_biology", "statistical_mechanics",
"classical_mechanics", "molecular_biology", "statistical_mechanics"
]


topic_data, topic_data_count = generate_topic_data_wikipedia(topics, 2000)

# CSV 파일로 데이터 저장
with open('clear_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['text', 'label'])  # 헤더 추가
    for text, label in topic_data:
        writer.writerow([text, label])

print(f"Total generated data samples: {len(topic_data)}")
print("Sample data:")
for i in range(min(5, len(topic_data))):
    print(f"Text: {topic_data[i][0]}")
    print(f"Label: {topic_data[i][1]}")
    print()

print("\nData count per topic:")
for topic, count in topic_data_count.items():
    print(f"{topic}: {count} samples")

print("\nData saved to topic_data.csv")

Total generated data samples: 3713
Sample data:
Text: Quantum mechanics is a fundamental theory that describes the behavior of nature at and below the scale of atoms.: 1.1  It is the foundation of all quantum physics, which includes quantum chemistry, quantum field theory, quantum technology, and quantum information science.
Quantum mechanics can describe many systems that classical physics cannot
Label: quantum_mechanics

Text: Classical physics can describe many aspects of nature at an ordinary (macroscopic and (optical) microscopic) scale, but is not sufficient for describing them at very small submicroscopic (atomic and subatomic) scales
Label: quantum_mechanics

Text: Most theories in classical physics can be derived from quantum mechanics as an approximation, valid at large (macroscopic/microscopic) scale.
Quantum systems have bound states that are quantized to discrete values of energy, momentum, angular momentum, and other quantities, in contrast to classical systems where thes

In [2]:
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")

('ratings_train.txt', <http.client.HTTPMessage at 0x17679c3beb0>)

In [ ]:
import csv
from collections import defaultdict

# 1. CSV 파일을 읽어서 데이터를 메모리에 로드
data_by_topic = defaultdict(list)

with open('clear_data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # 헤더 스킵
    for text, label in reader:
        data_by_topic[label].append(text)

# 2. 토픽별 샘플 수를 계산하고 정렬
topic_counts = {topic: len(samples) for topic, samples in data_by_topic.items()}
sorted_topics = sorted(topic_counts.items(), key=lambda x: x[1], reverse=True)

# 3. 상위 15개 토픽 선택
top_15_topics = dict(sorted_topics[:15])

# 4. 선택된 토픽의 데이터만 새로운 CSV 파일로 저장
output_filename = 'f_clear_topic.csv'

with open(output_filename, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['text', 'label'])  # 헤더 추가
    
    for topic in top_15_topics.keys():
        for text in data_by_topic[topic]:
            writer.writerow([text, topic])

# 5. 결과 출력
print("전체 토픽 수:", len(topic_counts))
print("\n모든 토픽의 샘플 수 (내림차순):")
for topic, count in sorted_topics:
    print(f"{topic}: {count} samples")

print("\n선택된 상위 15개 토픽:")
for topic, count in top_15_topics.items():
    print(f"{topic}: {count} samples")

print(f"\n총 선택된 샘플 수: {sum(top_15_topics.values())}")
print(f"데이터가 {output_filename}에 저장되었습니다.")

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# CSV 파일 읽기
df = pd.read_csv('f_clear_topic.csv', delimiter=',')

def preprocess_text(text):
    # 입력값이 문자열이 아닐 경우 빈 문자열로 처리
    if not isinstance(text, str):
        return ''
    # 소문자 변환
    text = text.lower()
    # 숫자 및 특수 문자 제거
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # 숫자 제거
    # 토큰화
    tokens = word_tokenize(text)
    # 불용어 제거 및 lemmatization
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    tokens = [
        lemmatizer.lemmatize(word) 
        for word in tokens 
        if word not in stop_words and len(word) > 1  # 두 글자 이상만 선택
    ]
    return ' '.join(tokens)

# 전처리 적용 (text 칼럼에만)
df['text'] = df['text'].apply(preprocess_text)

# 빈 행 제거
df = df[df['text'].str.strip() != '']

# 결과 확인
print(f"전처리된 샘플 수: {len(df)}")
print("\n처음 5개 전처리된 샘플:")
print(df.head())

# 전처리된 결과를 새 CSV 파일로 저장
df.to_csv('p_clear_topic.csv', index=False, header=True)

## Ambiguous Topic

In [ ]:
import requests
import csv

def get_wikipedia_content(topic):
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "titles": topic,
        "prop": "extracts",
        "explaintext": True,
        "redirects": 1
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    pages = data['query']['pages']
    for page_id, page_data in pages.items():
        return page_data.get('extract')

def generate_topic_data_wikipedia(topics, n_per_topic=2000):
    all_data = []
    topic_data_count = {}
    for topic in topics:
        content = get_wikipedia_content(topic)
        if content:
            sentences = content.split(". ")
            samples = [sentence for sentence in sentences if len(sentence.split()) >= 10]
            samples = samples[:n_per_topic]
            topic_data_count[topic] = len(samples)
            all_data.extend((sentence, topic) for sentence in samples)
        else:
            print(f"Failed to retrieve content for {topic}")
            topic_data_count[topic] = 0
    return all_data, topic_data_count

topics = [
"machine_learning", "deep_learning", "natural_language_processing", "computer_vision", "artificial_intelligence",
"neural_networks", "pattern_recognition", "data_mining", "big_data_analytics", "knowledge_discovery", "data_mining",
"big_data_analytics", "knowledge_discovery", "cognitive_computing", "reinforcement_learning", "intelligent_systems", "cognitive_computing", "reinforcement_learning", "intelligent_systems", "text_mining", "speech_recognition", "image_processing", "information_retrieval", "semantic_analysis", "computational_linguistics", "sentiment_analysis", "speech_recognition", "image_processing", "information_retrieval", "semantic_analysis", "computational_linguistics", "sentiment_analysis"
]


topic_data, topic_data_count = generate_topic_data_wikipedia(topics, 2000)

# CSV 파일로 데이터 저장
with open('ambi_topic.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['text', 'label'])  # 헤더 추가
    for text, label in topic_data:
        writer.writerow([text, label])

print(f"Total generated data samples: {len(topic_data)}")
print("Sample data:")
for i in range(min(5, len(topic_data))):
    print(f"Text: {topic_data[i][0]}")
    print(f"Label: {topic_data[i][1]}")
    print()

print("\nData count per topic:")
for topic, count in topic_data_count.items():
    print(f"{topic}: {count} samples")

print("\nData saved to topic_data.csv")

In [ ]:
import csv
from collections import defaultdict

# 1. CSV 파일을 읽어서 데이터를 메모리에 로드
data_by_topic = defaultdict(list)

with open('ambi_topic.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # 헤더 스킵
    for text, label in reader:
        data_by_topic[label].append(text)

# 2. 토픽별 샘플 수를 계산하고 정렬
topic_counts = {topic: len(samples) for topic, samples in data_by_topic.items()}
sorted_topics = sorted(topic_counts.items(), key=lambda x: x[1], reverse=True)

# 3. 상위 15개 토픽 선택
top_15_topics = dict(sorted_topics[:15])

# 4. 선택된 토픽의 데이터만 새로운 CSV 파일로 저장
output_filename = 'f_ambi_topic.csv'

with open(output_filename, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['text', 'label'])  # 헤더 추가
    
    for topic in top_15_topics.keys():
        for text in data_by_topic[topic]:
            writer.writerow([text, topic])

# 5. 결과 출력
print("전체 토픽 수:", len(topic_counts))
print("\n모든 토픽의 샘플 수 (내림차순):")
for topic, count in sorted_topics:
    print(f"{topic}: {count} samples")

print("\n선택된 상위 15개 토픽:")
for topic, count in top_15_topics.items():
    print(f"{topic}: {count} samples")

print(f"\n총 선택된 샘플 수: {sum(top_15_topics.values())}")
print(f"데이터가 {output_filename}에 저장되었습니다.")

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# CSV 파일 읽기
df = pd.read_csv('f_ambi_topic.csv', delimiter=',')

def preprocess_text(text):
    # 입력값이 문자열이 아닐 경우 빈 문자열로 처리
    if not isinstance(text, str):
        return ''
    # 소문자 변환
    text = text.lower()
    # 숫자 및 특수 문자 제거
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # 숫자 제거
    # 토큰화
    tokens = word_tokenize(text)
    # 불용어 제거 및 lemmatization
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    tokens = [
        lemmatizer.lemmatize(word) 
        for word in tokens 
        if word not in stop_words and len(word) > 1  # 두 글자 이상만 선택
    ]
    return ' '.join(tokens)

# 전처리 적용 (text 칼럼에만)
df['text'] = df['text'].apply(preprocess_text)

# 빈 행 제거
df = df[df['text'].str.strip() != '']

# 결과 확인
print(f"전처리된 샘플 수: {len(df)}")
print("\n처음 5개 전처리된 샘플:")
print(df.head())

# 토픽의 갯수 구하기
topic_count = df['label'].nunique()

print(f"토픽의 갯수: {topic_count}")


# 전처리된 결과를 새 CSV 파일로 저장
df.to_csv('p_ambi_topic.csv', index=False, header=True)

## More ambiguous Topics

In [ ]:
import requests
import csv

def get_wikipedia_content(topic):
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "titles": topic,
        "prop": "extracts",
        "explaintext": True,
        "redirects": 1
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    pages = data['query']['pages']
    for page_id, page_data in pages.items():
        return page_data.get('extract')

def generate_topic_data_wikipedia(topics, n_per_topic=2000):
    all_data = []
    topic_data_count = {}
    for topic in topics:
        content = get_wikipedia_content(topic)
        if content:
            sentences = content.split(". ")
            samples = [sentence for sentence in sentences if len(sentence.split()) >= 10]
            samples = samples[:n_per_topic]
            topic_data_count[topic] = len(samples)
            all_data.extend((sentence, topic) for sentence in samples)
        else:
            print(f"Failed to retrieve content for {topic}")
            topic_data_count[topic] = 0
    return all_data, topic_data_count

topics = [
    "Climate Change", "Global Warming", "Environmental Degradation", "Air Pollution", "Water Pollution",
    "Deforestation", "Biodiversity Loss", "Endangered Species", "Renewable Energy", "Fossil Fuel Dependency",
    "Social Inequality", "Income Inequality", "Gender Inequality", "Racial Inequality", "Educational Inequality",
    "Poverty Reduction", "Global Poverty", "Health Disparities", "Access to Healthcare", "Food Security",
    "Food Scarcity", "Water Scarcity", "Overpopulation", "Urbanization Challenges", "Housing Crisis"
]


topic_data, topic_data_count = generate_topic_data_wikipedia(topics, 2000)

# CSV 파일로 데이터 저장
with open('moreambi_topic.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['text', 'label'])  # 헤더 추가
    for text, label in topic_data:
        writer.writerow([text, label])

print(f"Total generated data samples: {len(topic_data)}")
print("Sample data:")
for i in range(min(5, len(topic_data))):
    print(f"Text: {topic_data[i][0]}")
    print(f"Label: {topic_data[i][1]}")
    print()

print("\nData count per topic:")
for topic, count in topic_data_count.items():
    print(f"{topic}: {count} samples")

print("\nData saved to topic_data.csv")

In [ ]:
import csv
from collections import defaultdict

# 1. CSV 파일을 읽어서 데이터를 메모리에 로드
data_by_topic = defaultdict(list)

with open('moreambi_topic.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # 헤더 스킵
    for text, label in reader:
        data_by_topic[label].append(text)

# 2. 토픽별 샘플 수를 계산하고 정렬
topic_counts = {topic: len(samples) for topic, samples in data_by_topic.items()}
sorted_topics = sorted(topic_counts.items(), key=lambda x: x[1], reverse=True)

# 3. 상위 15개 토픽 선택
top_15_topics = dict(sorted_topics[:15])

# 4. 선택된 토픽의 데이터만 새로운 CSV 파일로 저장
output_filename = 'f_moreambi_topic.csv'

with open(output_filename, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['text', 'label'])  # 헤더 추가
    
    for topic in top_15_topics.keys():
        for text in data_by_topic[topic]:
            writer.writerow([text, topic])

# 5. 결과 출력
print("전체 토픽 수:", len(topic_counts))
print("\n모든 토픽의 샘플 수 (내림차순):")
for topic, count in sorted_topics:
    print(f"{topic}: {count} samples")

print("\n선택된 상위 15개 토픽:")
for topic, count in top_15_topics.items():
    print(f"{topic}: {count} samples")

print(f"\n총 선택된 샘플 수: {sum(top_15_topics.values())}")
print(f"데이터가 {output_filename}에 저장되었습니다.")

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# CSV 파일 읽기
df = pd.read_csv('f_moreambi_topic.csv', delimiter=',')

def preprocess_text(text):
    # 입력값이 문자열이 아닐 경우 빈 문자열로 처리
    if not isinstance(text, str):
        return ''
    # 소문자 변환
    text = text.lower()
    # 숫자 및 특수 문자 제거
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # 숫자 제거
    # 토큰화
    tokens = word_tokenize(text)
    # 불용어 제거 및 lemmatization
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    tokens = [
        lemmatizer.lemmatize(word) 
        for word in tokens 
        if word not in stop_words and len(word) > 1  # 두 글자 이상만 선택
    ]
    return ' '.join(tokens)

# 전처리 적용 (text 칼럼에만)
df['text'] = df['text'].apply(preprocess_text)

# 빈 행 제거
df = df[df['text'].str.strip() != '']

# 결과 확인
print(f"전처리된 샘플 수: {len(df)}")
print("\n처음 5개 전처리된 샘플:")
print(df.head())

# 토픽의 갯수 구하기
topic_count = df['label'].nunique()

print(f"토픽의 갯수: {topic_count}")


# 전처리된 결과를 새 CSV 파일로 저장
df.to_csv('p_moreambi_topic.csv', index=False, header=True)

## 데이터 통계

In [3]:
import pandas as pd
import numpy as np

# 저장된 파일 불러오기
df = pd.read_csv('more_similar_topic.csv', delimiter=',')


# 필요한 라이브러리 임포트
from collections import Counter

# 텍스트 길이(단어 수) 및 평균 단어 길이 통계 추가
df['word_count'] = df['text'].apply(lambda x: len(x.split()))
df['avg_word_length'] = df['text'].apply(lambda x: np.mean([len(word) for word in x.split()]) if x else 0)

# 전체 단어 빈도 계산
all_words = ' '.join(df['text']).split()
word_freq = Counter(all_words)

# 토픽별 문서 수 계산
topic_counts = df['label'].value_counts()

# 통계 출력
print("총 문서 수:", len(df))
print("평균 단어 수:", df['word_count'].mean())
print("평균 단어 길이:", df['avg_word_length'].mean())
print("상위 10개 단어 빈도:", word_freq.most_common(10))
print("토픽별 문서 수:\n", topic_counts)

# 통계 요약을 DataFrame으로 정리하여 저장
stats_df = pd.DataFrame({
    'Total Documents': [len(df)],
    'Average Word Count': [df['word_count'].mean()],
    'Average Word Length': [df['avg_word_length'].mean()],
    'Top 10 Words': [word_freq.most_common(10)],
    'Topic Counts': [topic_counts.to_dict()]
})

# # CSV 파일로 저장
# stats_df.to_csv('text_data_statistics.csv', index=False)

# 상위 5개 데이터 출력
print("\n처음 5개 데이터:")
print(df.head())


총 문서 수: 3444
평균 단어 수: 21.48199767711963
평균 단어 길이: 7.066325047275514
상위 10개 단어 빈도: [('data', 1606), ('learning', 762), ('system', 512), ('model', 497), ('speech', 460), ('image', 449), ('recognition', 435), ('information', 427), ('big', 425), ('used', 410)]
토픽별 문서 수:
 label
big_data_analytics             506
speech_recognition             480
artificial_intelligence        365
sentiment_analysis             282
reinforcement_learning         268
deep_learning                  251
machine_learning               235
data_mining                    156
image_processing               152
computer_vision                149
knowledge_discovery            144
information_retrieval          136
natural_language_processing    112
pattern_recognition             85
text_mining                     67
cognitive_computing             56
Name: count, dtype: int64

처음 5개 데이터:
                                                text               label  \
0  big data primarily refers data set large compl...

In [ ]:
import pandas as pd
import numpy as np

# CSV 파일 읽기
df = pd.read_csv('pre_data.csv', delimiter=',')

# 전체 샘플 수와 목표 샘플 수 설정
total_samples = len(df)
target_samples = 1000

# 각 카테고리별 비율 계산
category_ratios = df['label'].value_counts(normalize=True)

# 각 카테고리별 샘플 수 계산 (비율 유지)
category_samples = (category_ratios * target_samples).round().astype(int)

# 반올림으로 인한 오차 보정
diff = target_samples - category_samples.sum()
if diff != 0:
    category_samples[category_samples.index[0]] += diff

# 층화 샘플링 수행
sampled_df = pd.DataFrame()
for category, n_samples in category_samples.items():
    category_df = df[df['label'] == category]
    sampled_category = category_df.sample(n=n_samples, random_state=42)
    sampled_df = pd.concat([sampled_df, sampled_category])

# 결과 확인
print(f"원본 데이터 크기: {len(df)}")
print(f"샘플링된 데이터 크기: {len(sampled_df)}")
print("\n원본 데이터의 카테고리 분포:")
print(df['label'].value_counts())
print("\n샘플링된 데이터의 카테고리 분포:")
print(sampled_df['label'].value_counts())

# 샘플링된 데이터를 새로운 CSV 파일로 저장
sampled_df.to_csv('sampled_topic_data.csv', index=False)
print("\n샘플링된 데이터가 'sampled_topic_data.csv'로 저장되었습니다.")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns

# CSV 파일 읽기
df = pd.read_csv('pre_data.csv')

# 기본 정보 출력
print(f"Total number of documents: {len(df)}")
print(f"Number of unique topics: {df['label'].nunique()}")

# 토픽별 문서 수
topic_counts = df['label'].value_counts()
print("\nNumber of documents per topic:")
print(topic_counts)

# 문장 길이 계산
df['sentence_length'] = df['text'].apply(lambda x: len(x.split()))

# 기초 통계 계산
print("\nBasic statistics of sentence length:")
print(df['sentence_length'].describe())

# 토픽별 평균 문장 길이
avg_sentence_length = df.groupby('label')['sentence_length'].mean().sort_values(ascending=False)
print("\nAverage sentence length per topic:")
print(avg_sentence_length)

# 가장 많이 사용된 단어 (불용어 제외)
def get_top_words(text, n=10):
    words = text.lower().split()
    word_counts = Counter(words)
    return word_counts.most_common(n)

top_words = get_top_words(' '.join(df['text']))
print("\nTop 10 most common words:")
print(top_words)

# 시각화: 토픽별 문서 수
plt.figure(figsize=(12, 6))
topic_counts.plot(kind='bar')
plt.title('Number of Documents per Topic')
plt.xlabel('Topic')
plt.ylabel('Number of Documents')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# 시각화: 문장 길이 분포
plt.figure(figsize=(12, 6))
sns.histplot(df['sentence_length'], kde=True)
plt.title('Distribution of Sentence Lengths')
plt.xlabel('Sentence Length (words)')
plt.ylabel('Frequency')


In [ ]:
import pandas as pd

# Load the preprocessed data
df = pd.read_csv('pre_data.csv')

# Get the unique labels and their counts
label_counts = df['label'].value_counts()

# Print the label names and their counts
print("Label counts:")
for label, count in label_counts.items():
    print(f"{label}: {count} samples")

In [3]:
import pandas as pd
import re
from konlpy.tag import Okt

# 데이터 파일 경로
file_path = "text.csv"

# 데이터 로드
data = pd.read_csv(file_path, sep=',')

# 불용어 목록 (예시)
stopwords = ['의', '가', '이', '은', '는', '에', '과', '도', '를', '을', '에서', '와', '한', '하다']

# 전처리 함수
def preprocess_text(text):
    # 특수 문자 제거
    text = re.sub(r'[^가-힣\s]', '', text)  # 한글과 공백만 남김
    # 형태소 분석
    okt = Okt()
    tokens = okt.morphs(text)  # 형태소 단위로 분리
    # 불용어 제거
    tokens = [word for word in tokens if word not in stopwords]
    return ' '.join(tokens)

# 'document' 컬럼에 대해 전처리 수행 (컬럼 이름을 실제 데이터에 맞게 수정하세요)
data['processed_document'] = data['document'].apply(preprocess_text)

# 전처리된 데이터를 새로운 CSV 파일로 저장
data['processed_document'].to_csv('p_text.csv', index=False)

print("전처리된 데이터가 p_text.csv에 저장되었습니다.")

전처리된 데이터가 p_text.csv에 저장되었습니다.


In [3]:
import pandas as pd

import pandas as pd

# CSV 파일 경로 (여기서 'iris.txt'는 실제 경로로 수정해야 합니다)
file_path = 'iris.txt'  # 파일 이름은 'iris.txt'로 수정

# 데이터셋 불러오기
iris_df = pd.read_csv(file_path, header=None, names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species'])

# 데이터셋을 새로운 CSV 파일로 저장
iris_df.to_csv('iris_output.txt', index=False)  # 파일 이름을 'iris_output.csv'로 수정